<a href="https://colab.research.google.com/github/mkunyosi/FLAI-ML-DDS7/blob/learning/FLAI_7aComp_s06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FLAI - 7a Competição de ML**
jan/2022 - by Marcos Kunyosi

Base para sub 7 e 8

## Global constants

In [1]:
fn_train = 'https://raw.githubusercontent.com/mkunyosi/FLAI-ML-DDS7/learning/new_treino.csv'
fn_test = 'https://raw.githubusercontent.com/mkunyosi/FLAI-ML-DDS7/learning/new_teste.csv'

fn_train_no_missdata = 'https://raw.githubusercontent.com/mkunyosi/FLAI-ML-DDS7/learning/treino_no_missdata.csv'
fn_test_no_missdata = 'https://raw.githubusercontent.com/mkunyosi/FLAI-ML-DDS7/learning/teste_no_missdata.csv'


vars_all_categorical = ['sexo', 'faixa_etaria', 'cor_raca', 'escolaridade', 'portador_de_deficiencia', 'tamanho_da_empresa']
vars_all_numerical = ['idade', 'horas_trabalho_por_semana', 'tempo_no_emprego', 'id']
var_target = 'remuneracao'
vars_all = vars_all_categorical + vars_all_numerical



In [2]:
vars_all

['sexo',
 'faixa_etaria',
 'cor_raca',
 'escolaridade',
 'portador_de_deficiencia',
 'tamanho_da_empresa',
 'idade',
 'horas_trabalho_por_semana',
 'tempo_no_emprego',
 'id']

## General Libraries

### General installs
*   PyCaret - AutoML </br>
*   Sweetviz


In [3]:

!pip install pycaret

!pip install sweetviz 


     |████████████████████████████████| 301 kB 4.2 MB/s 
     |████████████████████████████████| 86 kB 5.4 MB/s 
     |████████████████████████████████| 167 kB 49.5 MB/s 
     |████████████████████████████████| 15.6 MB 48.5 MB/s 
     |████████████████████████████████| 2.0 MB 45.3 MB/s 
     |████████████████████████████████| 261 kB 49.5 MB/s 
     |████████████████████████████████| 1.3 MB 32.5 MB/s 
     |████████████████████████████████| 6.8 MB 38.7 MB/s 
     |████████████████████████████████| 56 kB 4.3 MB/s 
     |████████████████████████████████| 1.7 MB 48.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 114 kB 38.6 MB/s 
     |████████████████████████████████| 303 kB 47.6 MB/s 
     |████████████████████████████████| 636 kB 42.0 MB/s 
     |████████████████████████████████| 675 kB 43.9 MB/s 
     |█████████████████

### General imports

In [4]:
# Importing data using pandas
import pandas as pd

# importing pycaret - AutoML
from pycaret.datasets import get_data

# importing visualization libs
import sweetviz as sv
import missingno as missing

#KNN imputer for missing data
import numpy as np
from sklearn.impute import KNNImputer

#https://stackoverflow.com/questions/57086398/how-do-you-clear-google-colabs-output-periodically
from google.colab import output

from pycaret.regression import * 


/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


## Functions

### General purpose

In [5]:
import inspect, re

def varname(p):
  for line in inspect.getframeinfo(inspect.currentframe().f_back)[3]:
    m = re.search(r'\bvarname\s*\(\s*([A-Za-z_][A-Za-z0-9_]*)\s*\)', line)
    if m:
      return m.group(1)

# spam = 42
# print(varname(spam))


In [6]:
#Verificando os valores possiveis das Features do Dataset de Treino
def ver_uniqueValues(df, pName):
    print(f"{pName}: ")
    for coluna in df.columns:
        print(coluna)
        print(df.sort_values(by=coluna)[coluna].unique())
        print("-"*100)
    print("="*100)

### Reading dataset

In [7]:
def read_dataset(train_file, test_file):
    #importing data tables from public folder (github)
    # df_train_raw =  pd.read_csv('https://raw.githubusercontent.com/mkunyosi/FLAI-ML-DDS7/learning/new_treino.csv')
    # df_test_raw =  pd.read_csv('https://raw.githubusercontent.com/mkunyosi/FLAI-ML-DDS7/learning/new_teste.csv')

    df_train_raw =  pd.read_csv(train_file) 
    df_test_raw =  pd.read_csv(test_file)   

    return df_train_raw, df_test_raw


In [8]:
# for preserving original data tables while coding is not finished
# data for handling is copied to other variables
# that way it is possible easily to get to the original data and run other experiments

#df_data_train = data_train_raw
#df_data_test = data_test_raw
def recover_originalData(df_train_raw, df_test_raw):
    #copy data tables for handling in next steps
    #df_data_train = data_train_raw.copy()
    #df_data_test = data_test_raw.copy()

    train = df_train_raw.copy()
    test = df_test_raw.copy()

    return train, test

#df_data_train, df_data_test = recover_originalData()


### Processing data for this specific job

#### df_preprocessing

In [9]:
if False:
    #tt['tempo_no_emprego']=df_data_train['tempo_no_emprego']#.apply(lambda x: round(x/6,1)+1)
    tt['idade'].plot(kind = 'hist')
    #tt['tempo_no_emprego'].plot()
    df_data_train

In [10]:
import math

def df_preprocessing(df, drop_vars, cut_data, interactive=False):

    global target_mean, target_std, target_min

    df['sexo'] = df['sexo'].apply(lambda x: 'm' if x == 1 else 'f')
    df['escolaridade'] = df['escolaridade'].apply(lambda x: f'_{x}' )
    df['faixa_etaria'] = df['faixa_etaria'].apply(lambda x: f'_{x}' )
    df['portador_de_deficiencia'] = df['portador_de_deficiencia'].apply(lambda x: f'_{x}' )
    df['tamanho_da_empresa'] = df['tamanho_da_empresa'].apply(lambda x: f'_{x}' )
    
    #df['tempo_no_emprego'] = df['tempo_no_emprego'].apply(lambda x: x/12)
    #df['tempo_no_emprego'] = df['tempo_no_emprego'].apply(lambda x: round(x/12)+1)
    #df['tempo_no_emprego'] = df['tempo_no_emprego'].apply(lambda x: int(x) )
    df['tempo_no_emprego'] = df['tempo_no_emprego'].apply(lambda x: x/600 )
    #df['tempo_no_emprego'] = df['tempo_no_emprego'].apply(lambda x: 0 if x == 0 else math.log(x) )

    df['idade'] = df['idade'].apply(lambda x: x/60 )
    
    
    # set missing values
    df['cor_raca'] = df['cor_raca'].map(lambda x: f'c_{x}' if x != 99 else np.nan)


    if True:
        if 'remuneracao' in df.columns:
            # df['remuneracao'] = df['remuneracao'].apply(lambda x: x/1000)
            # print(f"df['remuneracao']\n{df['remuneracao'].head(10)}")

            #df['remuneracao'] = df['remuneracao'].map(lambda x: math.log(x))
            df['remuneracao'] = df['remuneracao']
            # target_mean = np.mean(df['remuneracao'])
            # target_std = np.std(df['remuneracao'])
            # #df['remuneracao_orig'] = df['remuneracao']
            # df['remuneracao'] = (df['remuneracao'] - target_mean)/target_std
            # target_min = df['remuneracao'].min()
            # if target_min < 0:
            #     df['remuneracao'] = (df['remuneracao'] - target_min)# + 0.1)
            # print(f"df['remuneracao'].min()={df['remuneracao'].min()}")
            # print(f"df['remuneracao'].max()={df['remuneracao'].max()}")
            # print(f"target_mean={target_mean}\ntarget_std={target_std}")

    # drop vars
    print(f"[df_preprocessing]1. drop_vars={drop_vars}")
    df.drop(drop_vars, axis=1, inplace=True)
    print(f"[df_preprocessing]11. df.columns={df.columns}")


    if cut_data:
        #df = df.drop(df[df['tempo_no_emprego'] >= 100].index) #, axis=0, inplace=True)
        #df = df.drop(df[df['tempo_no_emprego'] < 100].index) #, axis=0, inplace=True)

        if 'remuneracao' in df.columns:
            #df = df.drop(df[(df['remuneracao'] < 15000) | (df['remuneracao'] > 30000)].index) #, axis=0, inplace=True)
            #df = df.drop(df[(df['remuneracao'] <= 15000) ].index) #, axis=0, inplace=True)
            df = df.drop(df[(df['remuneracao'] > 8000) ].index) #, axis=0, inplace=True)

        df.reset_index(inplace=True)
        df.drop(['index'], axis=1, inplace=True)    

    
    print(f"df = {df.shape}")
    if interactive:
        input('type something to continue')

    input('type something to continue')

    return df

In [11]:
def df_preprocessing_forMKModel(df):

    global target_mean, target_std, target_min

    # df['sexo'] = df['sexo'].apply(lambda x: 'm' if x == 1 else 'f')
    # df['escolaridade'] = df['escolaridade'].apply(lambda x: f'_{x}' )
    # df['faixa_etaria'] = df['faixa_etaria'].apply(lambda x: f'_{x}' )
    # df['portador_de_deficiencia'] = df['portador_de_deficiencia'].apply(lambda x: f'_{x}' )
    # df['tamanho_da_empresa'] = df['tamanho_da_empresa'].apply(lambda x: f'_{x}' )
    #df['tempo_no_emprego'] = df['tempo_no_emprego'].apply(lambda x: x/600 )

    df['sexo'] = df['sexo'].apply(lambda x: int(x))
    df['escolaridade'] = df['escolaridade'].apply(lambda x: int(x))
    df['faixa_etaria'] = df['faixa_etaria'].apply(lambda x: int(x))
    df['portador_de_deficiencia'] = df['portador_de_deficiencia'].apply(lambda x: int(x))
    df['tamanho_da_empresa'] = df['tamanho_da_empresa'].apply(lambda x: int(x))
    df['tempo_no_emprego'] = df['tempo_no_emprego'].apply(lambda x: int(x) )
    
    # set missing values
    #df['cor_raca'] = df['cor_raca'].map(lambda x: f'c_{x}' if x != 99 else np.nan)

    return df

In [12]:
#### df_replaceMissingData

In [13]:
def df_replaceMissingData(df_train, df_test, p_neighbor_vars, verbose=False):
#, drop_vars):    
    #global fn_train, fn_test
    # global data_train_raw, data_test_raw    
    # global df_data_train, df_data_test
    #global ensemble_model
    #global pred, preds_test        

    #-----------------------------------------------------------------
    # data_train_raw, data_test_raw = read_dataset(fn_train, fn_test)
    # df_data_train, df_data_test = recover_originalData(data_train_raw, data_test_raw)

    # df_data_train = df_train
    # df_data_test = df_test

    #--------------------------------------------------------------------
    
    # df_data_train = df_preprocessing(df_data_train,drop_vars, cut_data = False, interactive=False)
    # df_data_test = df_preprocessing(df_data_test, drop_vars, cut_data = False, interactive=False)
        
    #df.drop(drop_vars, axis=1, inplace=True)

    #-----------------------------------------------------------------------
    
    #*AQUI*neighbor_vars = ['tamanho_da_empresa', 'escolaridade', 'horas_trabalho_por_semana', 'tempo_no_emprego'] #, 'idade', 'tamanho_da_empresa', 'sexo']

    #df_trn = df_data_train[neighbor_vars].copy()
    df_trn = df_train[p_neighbor_vars].copy()

    #df_trn["cor_raca"] = df_data_train["cor_raca"]
    df_trn["cor_raca"] = df_train["cor_raca"].map(lambda x: x if x != 99 else np.nan)
    df_trn["cor_raca_new"] = df_trn["cor_raca"] #.map(lambda x: np.nan if x == 99 else x)
    df_trn["orig"] = df_trn["cor_raca"].map(lambda x: 'train')
    df_trn["orig_idx"] = range(df_trn.shape[0])

    #----------------
    #df_tst = df_data_test[neighbor_vars].copy()
    df_tst = df_test[p_neighbor_vars].copy()

    #df_tst["cor_raca"] = df_data_test["cor_raca"]
    df_tst["cor_raca"] = df_test["cor_raca"].map(lambda x: x if x != 99 else np.nan)
    df_tst["cor_raca_new"] = df_tst["cor_raca"] #.map(lambda x: np.nan if x == 99 else x)
    df_tst["orig"] = df_tst["cor_raca"].map(lambda x: 'test')
    df_tst["orig_idx"] = range(df_tst.shape[0])
    df_tst_nan = df_tst[df_tst['cor_raca_new'].isna()].copy()

    df_eval = df_trn.copy()
    df_eval = df_eval.append(df_tst_nan)
    df_eval = df_eval.reset_index()

    df_eval = mk_imputer(df_eval, 'cor_raca_new', p_neighbor_vars, calcFunc='mode', verboseMode=False, keepLog=True)
    df_eval

    for i in range(df_eval.shape[0]):
        if df_eval.loc[i, 'orig'] == 'train':
            #df_trn.loc[df_eval.loc[i, 'orig_idx'], 'cor_raca_new'] = df_eval.loc[i, 'cor_raca_new']
            #df_data_train.loc[df_eval.loc[i, 'orig_idx'], 'cor_raca'] = df_eval.loc[i, 'cor_raca_new']
            df_train.loc[df_eval.loc[i, 'orig_idx'], 'cor_raca'] = int(df_eval.loc[i, 'cor_raca_new'])

            if i % 1000 == 0 and verbose == True:
                print(f"updating train ({i}) row...")

        elif df_eval.loc[i, 'orig'] == 'test':
            #df_tst.loc[df_eval.loc[i, 'orig_idx'], 'cor_raca_new'] = df_eval.loc[i, 'cor_raca_new']
            #df_data_test.loc[df_eval.loc[i, 'orig_idx'], 'cor_raca'] = df_eval.loc[i, 'cor_raca_new']
            df_test.loc[df_eval.loc[i, 'orig_idx'], 'cor_raca'] = int(df_eval.loc[i, 'cor_raca_new'])


            if i % 1000 == 0 and verbose == True:
                print(f"updating test ({i}) row...")


        else:
            print(f"Error: ({i})")


    # df_data_train = 
    # df_data_test=

    #-----

    return df_train, df_test


In [14]:
def df_exportCSV(df, fName, bDownload=True):
    
    subm = pd.DataFrame(df)
    #fName = 'no_missing_' + 'fn_train.csv'
    print(f"Exporting CSV file: {fName}")
    subm.to_csv(fName, index = False)

    if bDownload:
        from google.colab import files
        files.download(fName)


In [15]:
# obsolete
# function: data_transform
#   Pipiline form input data (train and test)
#   - dummification
#   - imputation
# sample:
#   p_df = dados
#   cat_vars = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6']
# return:
#   new  dataframe
def data_transform(p_df, p_cat_vars, k, var_target):
    # ref.: A Guide To KNN Imputation - by Kyaw Saw Htoon
    #   https://medium.com/@kyawsawhtoon/a-guide-to-knn-imputation-95e2dc496e
    
    # create categorical vars - dummification
    cat_variables = p_df[p_cat_vars].copy()
    #for i in p_cat_vars:
        #cat_variables[i] = cat_variables[i].map(lambda col: f"_{col}")
    #    cat_variables[i] = cat_variables[i].astype('object')
    cat_dummies = pd.get_dummies(cat_variables, drop_first=True) #, dtype=float)    
    #print("dummies:")
    #print(cat_dummies.info())
    
    # drop original categorical vars and create a new data frame
    p_df = p_df.drop(p_cat_vars, axis=1)      
    b_remove_target = False
    #if 'remuneracao' in p_df.columns:        
    if var_target in p_df.columns:        
        #target = p_df['remuneracao']
        target = p_df[var_target]
        #print(f"target-01:{target.isnull().mean()}")
        #p_df = p_df.drop(['remuneracao'], axis=1)        
        p_df = p_df.drop([var_target], axis=1)        
        b_remove_target = True
    #print("drop:")
    #print(p_df.info())
    p_df = pd.concat([p_df, cat_dummies], axis=1)
    #print("drop+dummies:")
    #print(p_df.info())

    # run KNNImputer for fulfilling missing values
    #imputer = KNNImputer(n_neighbors=5)
    imputer = KNNImputer(n_neighbors=k)
    p_df2 = pd.DataFrame(imputer.fit_transform(p_df), columns = p_df.columns)
    #print("imputer:")
    #print(p_df2.info())

    if b_remove_target:
        #print(p_df2.shape)
        #print(target.shape)
        #print(f"target-02:{target.isnull().mean()}")
        #p_df2['remuneracao'] = p_df2['sexo'].map(lambda x: target[x])
        #p_df2['remuneracao'] = target
        p_df2[var_target] = target

    for i in p_cat_vars:
        p_df2[i] = cat_variables[i]
   
    return p_df2

#### data_transformDummy

In [16]:
# function: data_transformDummy
#   Pipiline form input data (train and test)
#   - dummification
#   - imputation
# sample:
#   p_df = dados
#   cat_vars = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6']
# return:
#   new  dataframe
def data_transformDummy(p_df, p_cat_vars): #, k, var_target):
    # ref.: A Guide To KNN Imputation - by Kyaw Saw Htoon
    #   https://medium.com/@kyawsawhtoon/a-guide-to-knn-imputation-95e2dc496e
    
    # create categorical vars - dummification
    print(f"[data_transformDummy]1. cat_variables={p_cat_vars}")
    print(f"[data_transformDummy]1. p_df.columns={p_df.columns}")
    l_cat_vars = p_cat_vars.copy()
    for i in p_cat_vars:
        #print(f"[data_transformDummy]2.test({i})")
        if i not in p_df.columns:
            print(f"[data_transformDummy]3. remove({i})")
            l_cat_vars.remove(i)    
    print(f"[data_transformDummy]11. cat_variables={l_cat_vars}")

    cat_variables = p_df[l_cat_vars].copy()
    #for i in p_cat_vars:
        #cat_variables[i] = cat_variables[i].map(lambda col: f"_{col}")
    #    cat_variables[i] = cat_variables[i].astype('object')
    cat_dummies = pd.get_dummies(cat_variables, drop_first=True) #, dtype=float)    
    print(f"[data_transformDummy]21. p_df(5):\n{p_df.head(5)}")
    print(f"[data_transformDummy]31. dummies:\n{cat_dummies.info()}")
        
    # drop original categorical vars and create a new data frame
    p_df = p_df.drop(l_cat_vars, axis=1)  


    # b_remove_target = False
    # #if 'remuneracao' in p_df.columns:        
    # if var_target in p_df.columns:        
    #     #target = p_df['remuneracao']
    #     target = p_df[var_target]
    #     #print(f"target-01:{target.isnull().mean()}")
    #     #p_df = p_df.drop(['remuneracao'], axis=1)        
    #     p_df = p_df.drop([var_target], axis=1)        
    #     b_remove_target = True
    # #print("drop:")
    # #print(p_df.info())
    p_df = pd.concat([p_df, cat_dummies], axis=1)
    # #print("drop+dummies:")
    # #print(p_df.info())

    # # run KNNImputer for fulfilling missing values
    # #imputer = KNNImputer(n_neighbors=5)
    # imputer = KNNImputer(n_neighbors=k)
    # p_df2 = pd.DataFrame(imputer.fit_transform(p_df), columns = p_df.columns)
    # #print("imputer:")
    # #print(p_df2.info())

    # if b_remove_target:
    #     #print(p_df2.shape)
    #     #print(target.shape)
    #     #print(f"target-02:{target.isnull().mean()}")
    #     #p_df2['remuneracao'] = p_df2['sexo'].map(lambda x: target[x])
    #     #p_df2['remuneracao'] = target
    #     p_df2[var_target] = target

    # for i in p_cat_vars:
    #     p_df2[i] = cat_variables[i]
    p_df2 = p_df
   
    return p_df2

In [17]:
def updateLocalData(df, recItemsChanged, listTarget, df_filtered, allIndexChanged, calcFunc, level, inc, verboseMode, sTrace):
#, listVars

    # set prefix for debugging
    txtPrint = "{:<" + str(level*4) + "}"
    txtPrint = txtPrint.format(" ")    

    if verboseMode: print(txtPrint, "update local data ", recItemsChanged)

    if len(recItemsChanged) > 0:
        if verboseMode: print(txtPrint, "")
        if verboseMode: print(txtPrint, "len(recItemsChanged)=", len(recItemsChanged))
        for j in recItemsChanged:
            df.loc[j, listTarget] = df_filtered.loc[j, listTarget]
            df.loc[j, 'trace'] = df_filtered.loc[j, 'trace']
            allIndexChanged.append(j)
            if verboseMode: print(txtPrint, "recItemsChanged=", j, ' newValue=', df.loc[j, listTarget])


    #set mean for other data
    if verboseMode: print(txtPrint, 'mean/mode calculus', 'type=', calcFunc, 'len(sample)=', len(df_filtered[listTarget]))
    
    if calcFunc == 'mean':
        #targetMean = pd.to_numeric(round(np.mean(df_filtered[listTarget+"-ref"]),0), downcast='integer') 
        targetMean = np.mean(df_filtered[listTarget+"-ref"])
    elif calcFunc == 'mode':        
        targetMean = df_filtered[df_filtered[listTarget].notna()][listTarget+"-ref"].mode()[0]        
                
    else:
        if verboseMode: print(txtPrint, "invalid function", calcFunc) 
        return -1


    for j in df_filtered['Index']:
        if calcFunc == 'mean':
            if pd.isna(df_filtered.loc[j, listTarget]):                                    
                df.loc[j, listTarget] = pd.to_numeric(targetMean, downcast='integer') #int(targetMean )
                df.loc[j, 'trace'] = ' idx=' + str(j) + 'mean=' + str(targetMean+ inc) + ' trace:' + sTrace
                allIndexChanged.append(j)

        elif calcFunc == 'mode':
            if pd.isna(df_filtered.loc[j, listTarget]):                
                df.loc[j, listTarget] = targetMean 
                #df.loc[j, 'trace'] = ' idx=' + str(j) + 'mode=' + targetMean + str(inc) + ' trace:' + sTrace
                df.loc[j, 'trace'] = ' idx=' + str(j) + 'mode=' + str(targetMean) + str(inc) + ' trace:' + sTrace
                allIndexChanged.append(j)


    if verboseMode: print(txtPrint, "targetMean=", targetMean, 'listTarget=',listTarget, ' index-upd=', allIndexChanged)

    return 0



In [18]:

# função recursiva determinar valores de campos não preenchidos
def reviewTarget(df, listVars, calcFunc='mean', level=0, inc=.1, verboseMode=False, sTrace=''):
    #Idade, Entrada, Salário, Sexo, Cidade, Experiência
    

    # set prefix for debugging
    txtPrint = "{:<" + str(level*4) + "}"
    txtPrint = txtPrint.format(" ")    
    
    # get target var
    listTarget = listVars[0]
    listVars.remove(listTarget)

    # start list of index for all row changed    
    allIndexChanged = []

    if verboseMode: print(txtPrint, "reviewTarget-begin", "level=", level, "vars=", listVars)        
    # set prefix for debugging
    txtPrint = "{:<" + str(level*4 + 1) + "}"
    txtPrint = txtPrint.format(" ")
    
    # check recursion base
    if df.shape[0] <= 1:
        if verboseMode: print(txtPrint, "recursion-base: few rows to eval", 'n_rows=', df.shape[0]) 
        if verboseMode: print(txtPrint, "reviewTarget-end", "level=", level, "vars=", listVars)  
        return -1, []
    # there are, at least, 2 rows => go ahead
    
    
    # check if there is least 1 row with null on target var and 1 row with values on target var
    if df[listTarget].isnull().sum() == 0 or df[listTarget].notnull().sum() == 0:
        if verboseMode: print(txtPrint, "recursion-base: no enough data to eval", 'n_nulls=', df[listTarget].isnull().sum(), 'n_values=',  df[listTarget].notnull().sum()) 
        if verboseMode: print(txtPrint, "reviewTarget-end", "level=", level, "vars=", listVars) 
        return -2, []
    # there are, at least, 2 rows (1 null and 1 with value) => go ahead


    if len(listVars) < 1:
        if verboseMode: print(txtPrint, "recursion-base: no var to filter data", 'listVars=', listVars) 
        if verboseMode: print(txtPrint, "reviewTarget-end", "level=", level, "vars=", listVars) 
        return 0, []

    # get first filter var from the list and sort it
    listItem = listVars[0]             
    listVars.remove(listItem)
    itemsSorted = np.sort(df[listItem].unique())

    localTrace = sTrace + '/' + listItem

    #localDataUpdated = False
    #check number of distinct items on the new var
    if itemsSorted.shape[0] < 1:
        #small list => don't need to eval each item
        #raise Exception("List of vars is too small. Review 'listVars' parameter.")
        if verboseMode: print(txtPrint, "recursion-base: no enough data to eval", 'listVars=', listVars) 
        if verboseMode: print(txtPrint, "reviewTarget-end", "level=", level, "vars=", listVars) 
        return -3, []
    
    #let´s eval each item on the filter var
    for i in itemsSorted:

        localTrace = localTrace + '(' + str(i) + ')'      
          
        df_filtered = df[df[listItem] == i].copy()         
        
        if verboseMode: print(txtPrint, "Working in new data_frame ", localTrace, ">", i)

        #call recursive function to eval next var
        if verboseMode: print(txtPrint, "recursion(begin)...", 'listVars=', listVars, 'item=', i)

        listVarsCpy = listVars.copy()                
        listVarsCpy.insert(0, listTarget)
        reviewTarget_ret, recItemsChanged = reviewTarget(df_filtered, listVarsCpy, calcFunc, level+1, inc + .1, verboseMode, localTrace)
        
        if verboseMode: print(txtPrint, "recursion(end): ", reviewTarget_ret, 'listVars=', listVars)

        if reviewTarget_ret >=0:
            #if updateLocalData(df, recItemsChanged, listTarget, df_filtered, allIndexChanged, calcFunc, level, inc, verboseMode, localTrace) >= 0:
            #    localDataUpdated = True
            updateLocalData(df, recItemsChanged, listTarget, df_filtered, allIndexChanged, calcFunc, level, inc, verboseMode, localTrace) >= 0
            # if verboseMode: print(txtPrint, "update local data ", recItemsChanged)
            # if len(recItemsChanged) > 0:
            #     if verboseMode: print(txtPrint, "")
            #     if verboseMode: print(txtPrint, "len(recItemsChanged)=", len(recItemsChanged))
            #     for j in recItemsChanged:
            #         df.loc[j, listTarget] = df_filtered.loc[j, listTarget]
            #         df.loc[j, 'trace'] = df_filtered.loc[j, 'trace']
            #         allIndexChanged.append(j)
            #         if verboseMode: print(txtPrint, "recItemsChanged=", j, ' newValue=', df.loc[j, listTarget])

            
        
            # #set mean for other data
            # if verboseMode: print(txtPrint, 'mean/mode calculus', 'type=', calcFunc, 'len(sample)=', len(df_filtered[listTarget]))
            # #targetMean = round(np.mean(df_filtered[listTarget]),0) #+ inc
            # if calcFunc == 'mean':
            #     targetMean = pd.to_numeric(round(np.mean(df_filtered[listTarget]),0), downcast='integer') #int(round(np.mean(df_filtered[listTarget]),0) )
            # elif calcFunc == 'mode':
            #     #targetMean = df_filtered[df_filtered['Projeto-new'].notna()]['Projeto-new'].mode()[0]
            #     targetMean = df_filtered[df_filtered[listTarget].notna()][listTarget].mode()[0]
            #     #print('targetMean', targetMean)
                     
            # else:
            #     if verboseMode: print(txtPrint, "invalid function", calcFunc) 
            #     return -4, []
            

            # for j in df_filtered['Index']:
            #     if calcFunc == 'mean':
            #         if pd.isna(df_filtered.loc[j, listTarget]):                                    
            #             df.loc[j, listTarget] = pd.to_numeric(targetMean, downcast='integer') #int(targetMean )
            #             df.loc[j, 'trace'] = ' idx=' + str(j) + 'mean=' + str(targetMean+ inc) + ' trace:' + sTrace
            #             allIndexChanged.append(j)
            #     elif calcFunc == 'mode':
            #         if pd.isna(df_filtered.loc[j, listTarget]):                
            #             df.loc[j, listTarget] = targetMean 
            #             #df.loc[j, 'trace'] = ' idx=' + str(j) + 'mode=' + targetMean + str(inc) + ' trace:' + sTrace
            #             df.loc[j, 'trace'] = ' idx=' + str(j) + 'mode=' + str(targetMean) + str(inc) + ' trace:' + sTrace
            #             allIndexChanged.append(j)



            # if verboseMode: print(txtPrint, "targetMean=", targetMean, 'listTarget=',listTarget, ' index-upd=', allIndexChanged)


    #if localDataUpdated == False:
    df_filtered = df.copy()
    updateLocalData(df, recItemsChanged, listTarget, df_filtered, allIndexChanged, calcFunc, level, inc, verboseMode, localTrace)

    if verboseMode: print(txtPrint, "reviewTarget-end", "level=", level, "vars=", listVars) 

    return 0, allIndexChanged


In [19]:

def mk_imputer(df, target_var, neighbor_vars, calcFunc='mode', verboseMode=False, keepLog=False):
    
    #auxiliary vars
    df[target_var + "-ref"] = df[target_var]
    df['Index'] = range(df.shape[0])

    param_vars = neighbor_vars.copy()
    param_vars.insert(0, target_var)


    retCode, rowsChanged = reviewTarget(
                            df, 
                            param_vars, 
                            calcFunc, 
                            verboseMode
                            )
    
    if not keepLog:
        df = df.drop([target_var + "-ref", 'Index', 'trace'], axis=1, inplace=True)

    return df




### Running PyCaret

In [20]:
# setup
def pycaret_setup(df, target_var):
    print(f"df.shape={df.shape}")
    # pc_regression = setup(data = df_data_train, 
    #                   silent=True,
    #               target = target_var)
    pc_setup = setup(data = df, 
                    silent=False,
                    target = target_var)
    
    return pc_setup

In [21]:
# Comparing available models sorted by MAE metric (Mean Average Error)
# Selecting top 5 models to continue the evaluation
def pycaret_compareModels(topN):
    #top5_models = compare_models(sort = 'MAE', n_select = 5)
    
    bAllTries = True
    if bAllTries:
        top_models = compare_models(sort = 'MAE', n_select = topN)

    if not bAllTries:
        huber = create_model('huber')
        #br = create_model('br')
        par = create_model('par')
        #dt = create_model('dt')
        lightgbm = create_model('lightgbm')
        gbr = create_model('gbr')
        #top_models = [huber, br, dt]#, gbr]
        top_models = [huber, par, lightgbm, gbr]

    return top_models

In [22]:
# Tuning top 5 models
def pycaret_tunning(top_models):
    
    # top5_models_tuned = []
    # for i in range(len(top5_models)):
    #     #if i not in [2]:
    #     model_name = str(top5_models[i])
    #     model_name = model_name.split(sep='(')[0]
    #     print(f"Analysing: {model_name}")
    #     if model_name not in ['RandomForestRegressor', 'ExtraTreesRegressor']:
    #         # 3rd model = Random Forest Regressor (rf)
    #         # 4th model = Extra Trees Regressor (et)
    #         top5_models_tuned.append(tune_model(top5_models[i], optimize = 'MAE'))

    top_models_tuned = []
    for i in range(len(top_models)):
        model_name = str(top_models[i])
        model_name = model_name.split(sep='(')[0]
        print(f"Analysing: {model_name}")
        if model_name not in ['RandomForestRegressor', 'ExtraTreesRegressor']:
            top_models_tuned.append(tune_model(top_models[i], optimize = 'MAE'))


    return top_models_tuned

In [23]:
def pycaret_ensembleModel():
    global df_data_train, df_data_test
    # setup
    pc_regression = pycaret_setup(df_data_train, var_target)

    # Comparing available models sorted by MAE metric (Mean Average Error)
    # Selecting top 5 models to continue the evaluation
    top5_models = pycaret_compareModels(pycaret_N_models)

    #return []

    # Tuning top 5 models
    top5_models_tuned = pycaret_tunning(top5_models)

    # Mixing top 5 models
    ensemble_model = blend_models(top5_models_tuned)

    return ensemble_model


In [24]:
import random 

def simulate_eval(pred):
    sub_pred = pred.copy()
    #sub_pred = sub_pred[sub_pred['tempo_no_emprego'] < 100]
    #sub_pred.reset_index(inplace = True)

    n_rows = sub_pred.shape[0]
    rows = range(n_rows)
    simulations = 1000

    resultados = []
    for i in range(simulations):
        eval_list = random.sample(rows, k = int(n_rows/3))
        #eval_list = random.sample(rows, k = 10)
        #eval_list

        #abs(pred['Label']-pred['remuneracao']).mean()
        #pred['Label'][eval_list]
        m_mae = abs(sub_pred['Label'][eval_list]-sub_pred['remuneracao'][eval_list]).mean()
        resultados.append(m_mae)

    ev_mean = np.array(resultados).mean()
    ev_std = np.array(resultados).std()

    return ev_mean, ev_std

 



### Running cases

#### run_allSteps

#### eval_cases

In [25]:
def run_allSteps(p_train_vars, p_cutLevel, p_partDown):
    global fn_train, fn_test
    global data_train_raw, data_test_raw
    global df_data_train, df_data_test
    global ensemble_model
    global pred, preds_test
    #global target_var
    global var_target
    #-----------------------------------------------------------------    
    #data_train_raw, data_test_raw = read_dataset(fn_train, fn_test)
    data_train_raw, data_test_raw = read_dataset(fn_train_no_missdata, fn_test_no_missdata)
    
    df_data_train, df_data_test = recover_originalData(data_train_raw, data_test_raw)

    #--------------------------------------------------------------------
    # df_data_train = df_preprocessing(df_data_train, True, interactive=True)
    # df_data_test = df_preprocessing(df_data_test, True, interactive=True)
    
    #*AQUI*drop_vars = ['id']
    print(f"[run_allSteps]1. p_train_vars={p_train_vars}")
    l_drop_vars = vars_all.copy()
    print(f"[run_allSteps]6. l_drop_vars(all_vars)={l_drop_vars}")
    for i in p_train_vars: 
        l_drop_vars.remove(i)
    print(f"[run_allSteps]11. l_drop_vars={l_drop_vars}")

    df_data_train = df_preprocessing(df_data_train, l_drop_vars, cut_data = False, interactive=False)
    df_data_test = df_preprocessing(df_data_test, l_drop_vars, cut_data = False, interactive=False)

    print(f"[run_allSteps]21. df_data_train.columns={df_data_train.columns}")
        
    #-----------------------------------------------------------------------
    # run transformation for train and test data
    # df_data_train = data_transform(df_data_train, cat_vars, k_knnimputer, target_var)
    # df_data_test = data_transform(df_data_test, cat_vars, k_knnimputer, target_var)
    
    # #*AQUI*neighbor_vars = ['tamanho_da_empresa', 'escolaridade', 'horas_trabalho_por_semana', 'tempo_no_emprego'] #, 'idade', 'tamanho_da_empresa', 'sexo']

    # df_trn = df_data_train[neighbor_vars].copy()

    # df_trn["cor_raca"] = df_data_train["cor_raca"]
    # df_trn["cor_raca_new"] = df_trn["cor_raca"] #.map(lambda x: np.nan if x == 99 else x)
    # df_trn["orig"] = df_trn["cor_raca"].map(lambda x: 'train')
    # df_trn["orig_idx"] = range(df_trn.shape[0])

    # #----------------
    # df_tst = df_data_test[neighbor_vars].copy()

    # df_tst["cor_raca"] = df_data_test["cor_raca"]
    # df_tst["cor_raca_new"] = df_tst["cor_raca"] #.map(lambda x: np.nan if x == 99 else x)
    # df_tst["orig"] = df_tst["cor_raca"].map(lambda x: 'test')
    # df_tst["orig_idx"] = range(df_tst.shape[0])
    # df_tst_nan = df_tst[df_tst['cor_raca_new'].isna()].copy()

    # df_eval = df_trn.copy()
    # df_eval = df_eval.append(df_tst_nan)
    # df_eval = df_eval.reset_index()

    # df_eval = mk_imputer(df_eval, 'cor_raca_new', neighbor_vars, calcFunc='mode', verboseMode=False, keepLog=True)
    # df_eval

    # for i in range(df_eval.shape[0]):
    #     if df_eval.loc[i, 'orig'] == 'train':
    #         #df_trn.loc[df_eval.loc[i, 'orig_idx'], 'cor_raca_new'] = df_eval.loc[i, 'cor_raca_new']
    #         #df_data_train.loc[df_eval.loc[i, 'orig_idx'], 'cor_raca'] = df_eval.loc[i, 'cor_raca_new']
    #         df_data_train.loc[df_eval.loc[i, 'orig_idx'], 'cor_raca'] = df_eval.loc[i, 'cor_raca_new']
    #     elif df_eval.loc[i, 'orig'] == 'test':
    #         #df_tst.loc[df_eval.loc[i, 'orig_idx'], 'cor_raca_new'] = df_eval.loc[i, 'cor_raca_new']
    #         #df_data_test.loc[df_eval.loc[i, 'orig_idx'], 'cor_raca'] = df_eval.loc[i, 'cor_raca_new']
    #         df_data_test.loc[df_eval.loc[i, 'orig_idx'], 'cor_raca'] = df_eval.loc[i, 'cor_raca_new']
    #     else:
    #         print(f"Error: ({i})")


    # # df_data_train = 
    # # df_data_test=

    #-----
    #*AQUI*cat_vars = ['escolaridade', 'sexo', 'cor_raca']
    df_data_train = data_transformDummy(df_data_train, cat_vars) #, k_knnimputer, target_var)
    df_data_test = data_transformDummy(df_data_test, cat_vars)

    print(f"[run_allSteps]31. df_data_train.columns={df_data_train.columns}")

    if p_partDown:
        #df_data_train = df_data_train.drop(df_data_train[(df_data_train['remuneracao'] <= 8000) ].index) 
        #df_data_train = df_data_train.drop(df_data_train[(df_data_train['remuneracao'] > 7000) ].index) 
        df_data_train = df_data_train.drop(df_data_train[(df_data_train['remuneracao'] > p_cutLevel) ].index) 
    else:
        df_data_train = df_data_train.drop(df_data_train[(df_data_train['remuneracao'] <= p_cutLevel) ].index) 

    print(f"df_data_train.shape={df_data_train.shape}")
    print(f"df_data_train.columns={df_data_train.columns}")
    print(f"df_data_test.shape={df_data_test.shape}")
    print(f"df_data_test.columns={df_data_test.columns}")

    for i in df_data_train.columns:
        if (i not in df_data_test.columns) and (i not in var_target):
            df_data_test[i] = [0] * df_data_test.shape[0]
    
    print("---" * 10)
    print(f"df_data_train.shape={df_data_train.shape}")
    print(f"df_data_train.columns={df_data_train.columns}")
    print(f"df_data_test.shape={df_data_test.shape}")
    print(f"df_data_test.columns={df_data_test.columns}")

    
    #return 0, 0 , []

    #--------------------------------------------------------------------
    ensemble_model = pycaret_ensembleModel()

    return 0, 0 , []

    # Checking performance (MAE) using training data
    pred = predict_model(ensemble_model)

    # Predicting target values for test data
    preds_test = predict_model(ensemble_model, data = df_data_test)

    eval_mean, eval_std = simulate_eval(pred);
    #eval_model_params = ensemble_model.named_estimators
    eval_model_params = ensemble_model

    evaluate_model(ensemble_model)
    #input('type something to continue')


    return eval_mean, eval_std, eval_model_params

In [26]:
if False:
    dup = df_data_test
    # dup = df_data_train.drop('remuneracao', axis = 1)
    # dup = dup.append(df_data_test)
    dup = dup.duplicated()
    df_data_test[dup]


In [27]:
def eval_cases(p_names, p_tries, p_tryName):
    global fn_train, fn_test
    global data_train_raw, data_test_raw
    global df_data_train, df_data_test
    global ensemble_model
    global pred, preds_test
    #global cat_vars, drop_vars, target_var, pycaret_N_models
    global cat_vars, drop_vars, var_target, pycaret_N_models
    
    #, neighbor_vars

    all_runs0 = {}
    for i in range(len(p_tries)):
        #neighbor_vars = p_tries[i]['neighbor_vars']
        #target_var = p_tries[i]['target_var']        
        #drop_vars = p_tries[i]['drop_vars']
        #k_knnimputer = p_tries[i]['k_knnimputer']

        cat_vars = p_tries[i]['cat_vars']        
        l_train_vars = p_tries[i]['train_vars']
        var_target = p_tries[i]['var_target']
        pycaret_N_models = p_tries[i]['pycaret_N_models']

        l_cutLevel = p_tries[i]['cut_level']
        l_partDown = p_tries[i]['part_down']
        

        #print("Running {}\ncat_vars={}\ndrop_vars={}\ntarget_var={}\nk_knnimputer={}\npycaret_N_models={}\n".format(p_tryName[i], cat_vars, drop_vars, target_var, k_knnimputer, pycaret_N_models))
        print("Running {}\ncat_vars={}\nvar_target={}\nl_train_vars={}\npycaret_N_models={}\n".format(p_tryName[i], cat_vars, var_target, l_train_vars, pycaret_N_models))

        #eval_mean, eval_std, eval_model_params = [1.1, 1.2, 1.3]
        #eval_mean, eval_std, eval_model_params = run_allSteps(l_train_vars)
        eval_mean, eval_std, eval_model_params = run_allSteps(l_train_vars, l_cutLevel, l_partDown)
        all_runs0[all_runs_tryName[i]] = [eval_mean, eval_std, eval_model_params]

    #output.clear()
    all_runs = pd.DataFrame(all_runs0, index = all_runs_names).T
    return all_runs


### Other

In [28]:
# insert a new column for "indexing" all other columns
def createColAll(df, cols):  
  df['all'] = df[df.columns[0]].map(lambda x: "")
  #for i in range(df.shape[0]):    
  print('Inserting a new columns ("all")', end = '')
  c_print = 4
  for i in df.index.values:    
    # print(i)
    # return 0
    if i % 500 == 0:        
        print(f", {i}", end = '')
        c_print += 1
        if c_print > 10: 
            c_print = 0
            print('\n')

    for j in cols:
      if j not in ["all", "index"]:
        df['all'][i] = df['all'][i] + str(df[j][i]) + "-"

  print('\n')
  return df['all']


## Global variables

In [29]:

pycaret_N_models = 5




## Predicting values <<< START HERE <<<

In [39]:
all_runs_names = ['mean', 'std', 'models']


all_runs_tries = [{'cat_vars': ['sexo', 'faixa_etaria', 'cor_raca', 'escolaridade', 'portador_de_deficiencia', 'tamanho_da_empresa'],
                    'train_vars': ['escolaridade', 'tamanho_da_empresa', 'tempo_no_emprego', 'sexo', 'idade', 'cor_raca'], 
                    'var_target': 'remuneracao',
                    'cut_level': 8000,
                    'part_down': False,  # gerar modelo para remuneracao > 8K
                    #'part_down': True,  # gerar modelo para remuneracao <= 8K
                    'pycaret_N_models': 5 # número de modelos pycaret para serem testados
                   }
                
                ]

all_runs_tryName = [
                    'dados para análise: rem > 8000; pycaret: top 5'
                    ]

all_runs = eval_cases(all_runs_names, all_runs_tries, all_runs_tryName)
all_runs

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,5421.0185,6.202604e+07,7875.6611,0.0436,0.4330,0.3551
1,3402.7789,2.659375e+07,5156.9127,0.0896,0.3186,0.2590
2,3974.7907,3.355505e+07,5792.6723,-0.0560,0.3623,0.2795
3,5217.1436,7.378966e+07,8590.0908,0.0267,0.4138,0.3231
4,4750.9077,5.347206e+07,7312.4594,-0.0276,0.4006,0.2972
5,3574.5485,2.654044e+07,5151.7410,0.0606,0.3143,0.2489
6,5061.3085,4.163203e+07,6452.2884,0.0154,0.4071,0.3676
7,5161.4472,6.994432e+07,8363.2723,0.0595,0.4167,0.3197
8,5046.8101,5.483177e+07,7404.8475,-0.0054,0.4203,0.3449
9,5197.9540,6.869862e+07,8288.4632,0.0300,0.4209,0.3248


,mean,std,models
dados para análise: rem > 8000; pycaret: top 5,0,0,[]


In [40]:
# > 8k
# Checking performance (MAE) using training data
pred = predict_model(ensemble_model)

# Predicting target values for test data
preds_test = predict_model(ensemble_model, data = df_data_test)

eval_mean, eval_std = simulate_eval(pred);
#eval_model_params = ensemble_model.named_estimators
eval_model_params = ensemble_model

evaluate_model(ensemble_model)
#input('type something to continue')


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,4802.5016,5.552522e+07,7451.5246,0.0459,0.3993,0.3173


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

### Predição para submissão

In [ ]:
# Predicting target values for test data
pred02 = predict_model(ensemble_model, data = df_data_test)

subm = pd.DataFrame(pred02)
fName = 'pred_07.csv'
subm.to_csv(fName, index = False)

from google.colab import files
files.download(fName)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>